# **IATI Datastore API data collection**

## **Dependencies**

In [2]:
import requests
import json
from tqdm.notebook import tqdm

from importlib.machinery import SourceFileLoader
CONSTANTS = SourceFileLoader("CONSTANTS", "../../config/CONSTANTS.py").load_module()
KEYS = SourceFileLoader("KEYS", "../../config/KEYS.py").load_module()


## **Define relevant features**

List of all elements found here: https://developer.iatistandard.org/api-details#api=datastore&operation=query 

Description of all elements found here: https://iatistandard.org/en/iati-standard/203/activity-standard/summary-table/

In [1]:
fl_list = CONSTANTS.IATI_ATTRIBUTES
fl_str = ",".join(fl_list)

NameError: name 'CONSTANTS' is not defined

## **Fetch data from IATI Datastore**

### Fetch as JSON

In [5]:
##########
# PARAMS #
##########
# Fill in the abbreviation for the document name and the IATI code to access the right data
# abbreviation and codes can be found in the ./CONSTANTS.py or you can print out CONSTANTS.ORGANIZATIONS

abbreviation = "giz-non-bmz"
code = "XM-DAC-5-52"

In [6]:
url = "https://api.iatistandard.org/datastore/activity/select"
query = {
    "q": f'reporting_org_ref:("{code}")',
    "start": 0, 
    "rows": 1000,  # max 1000 rows per page
    "fl": fl_str
}

headers = {
    "Ocp-Apim-Subscription-Key": KEYS.IATI_KEY
}

try:
    all_docs = []
    page = 0
    pbar = tqdm()

    max_pages = 1
    max_pages_avl = False
    cur_page = -1

    while True:
        if cur_page < max_pages or max_pages_avl == False:
            cur_page += 1
            print(f"Page: {page}")
            response = requests.get(url, params=query, headers=headers)
            
            if response.status_code == 200:
                data = response.json()
                docs = data.get("response", {}).get("docs", [])
                if docs:
                    all_docs.extend(docs)
                    query["start"] += len(docs)
                    page += 1
                    pbar.update(1)
                else:
                    break
            else:
                print("Error:", response.status_code)
                break
        else:
            break

    pbar.close()

    with open(f"../../src/responses/response_{abbreviation}.json", mode='w', encoding='utf-8') as file:
        json.dump(all_docs, file, ensure_ascii=False, indent=4)
        
    print("Data downloaded and stored in src/responses/!'")

except requests.exceptions.RequestException as e:
    print("Error:", e)

0it [00:00, ?it/s]

Page: 0
Page: 1
Data downloaded and stored in src!'
